# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later
import sys
import os
import json
import pandas as pd
sys.path.insert(1, '../../../../src/')
import gt_modules.evaluation as evaluation
from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-sport4-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series (Tennis explorative search") 

Consider the following exploratory information need:

> investigate tennis and find the main BGPs related to this sport. Compare players per countries, awards of the players and tournament.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q1426`| Roger Federer       | node |
| `wd:Q11459`| Serena Williams       | node |





Also consider

```
wd:Q1426 ?p ?obj .
```

is the BGP to retrieve all **properties of Roger Federer**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for tennis player

2. Identify the BGP for tennis tournament

3. Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) (the result set must be a list of four couples Grand Slam IRI and label).

4. Find the three different type of possible surfaces in a tennis court (the result set must be a list of three couples surface IRI and label).

5. Return the tournaments played in the 2018 Wimbledon Championships edition with the related winners (hint: men and women do not play together, there are two distinct tournament ) (the result set must be a list of elements with tournament IRI and label, and winner IRI and label).

6. Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

7. Consider the country for sport property of the tennis player (NO Country of citizenship). Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

8. Consider the country for sport property of the tennis player (NO Country of citizenship). Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

In [11]:
## startup the evaluation
# setup the file and create the empty json
ipname = "s4.ipynb"
pt = os.getcwd()+os.sep+ipname
evaluation.setup(pt)

The index of this workflow is: 0_4


## Task 1 
Identify the BGP for tennis player

In [12]:
# starting from Roger Federer I look at his properties
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
    wd:Q1426 ?pr ?obj.
    ?pr sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
Empty


In [13]:
# starting from Roger Federer I look at his occupations
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    wd:Q1426 wdt:P106 ?obj.
    ?obj sc:name ?name .
} 
LIMIT 100
"""

print("Results")
x = run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q10833314'), ('name', 'tennis player')]
1


In [14]:
### insert the result of TASK 1 in the file
og_uri = "http://www.wikidata.org/entity/Q10833314'"
og_name = "tennis player"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"1", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 0_4
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_4.json
JSON object updated


`wd:Q10833314` is the node of the tennis player in wikidata

## Task 2
Identify the BGP for tennis tournament

In [16]:
# in the first query I found P2522 that is victory. Use it and explore
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
    wd:Q1426 wdt:P2522 ?x .
    ?x <http://schema.org/name> ?name .
}
"""

print("Results")
x= run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q753300'), ('name', "2012 Wimbledon Championships – Gentlemen's Singles")]
[('x', 'http://www.wikidata.org/entity/Q757054'), ('name', "tennis at the 2008 Summer Olympics – men's doubles")]
[('x', 'http://www.wikidata.org/entity/Q1074654'), ('name', "2006 Wimbledon Championships – Men's Singles")]
[('x', 'http://www.wikidata.org/entity/Q384929'), ('name', "2005 Wimbledon Championships – Men's Singles")]
[('x', 'http://www.wikidata.org/entity/Q782420'), ('name', "2006 Australian Open – Men's Singles")]
[('x', 'http://www.wikidata.org/entity/Q612597'), ('name', '2007 Tennis Masters Cup')]
[('x', 'http://www.wikidata.org/entity/Q300024'), ('name', '2011 ATP World Tour Finals')]
[('x', 'http://www.wikidata.org/entity/Q14916861'), ('name', '2014 Davis Cup')]
[('x', 'http://www.wikidata.org/entity/Q782589'), ('name', "2010 Australian Open – Men's Singles")]
[('x', 'http://www.wikidata.org/entity/Q22956224'), ('name', "2017 Australian Open – Men's

In [17]:
# look for the instance of the results found above
queryString = """
SELECT DISTINCT ?inst ?name
WHERE { 
    wd:Q1426 wdt:P2522 ?x .
    ?x wdt:P31 ?inst.  
    ?inst <http://schema.org/name> ?name .
}
"""

print("Results")
x= run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q18536594'), ('name', 'Olympic sporting event')]
[('inst', 'http://www.wikidata.org/entity/Q27020041'), ('name', 'sports season')]
[('inst', 'http://www.wikidata.org/entity/Q46190676'), ('name', 'tennis event')]
[('inst', 'http://www.wikidata.org/entity/Q47345468'), ('name', 'tennis tournament edition')]
[('inst', 'http://www.wikidata.org/entity/Q270907'), ('name', 'ATP Finals')]
5


In [18]:
# look for the instance of the results found above
queryString = """
SELECT DISTINCT ?inst ?name
WHERE { 
    wd:Q1426 wdt:P2522 ?x .
    ?x wdt:P31/wdt:P279 ?inst.  
    ?inst <http://schema.org/name> ?name .
}
"""

print("Results")
x= run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q13406554'), ('name', 'sports competition')]
[('inst', 'http://www.wikidata.org/entity/Q18608583'), ('name', 'recurring sporting event')]
[('inst', 'http://www.wikidata.org/entity/Q27020041'), ('name', 'sports season')]
[('inst', 'http://www.wikidata.org/entity/Q27968055'), ('name', 'recurrent event edition')]
[('inst', 'http://www.wikidata.org/entity/Q16510064'), ('name', 'sporting event')]
[('inst', 'http://www.wikidata.org/entity/Q12826913'), ('name', 'season')]
[('inst', 'http://www.wikidata.org/entity/Q23807345'), ('name', 'competition')]
[('inst', 'http://www.wikidata.org/entity/Q26863790'), ('name', 'ATP tennis tournament')]
8


In [19]:
# look for the superclass of the class of ATP tennis tournament edition Q26863790
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
    wd:Q26863790 wdt:P279 ?x .
    ?x <http://schema.org/name> ?name .
}
"""

print("Results")
x= run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q13219666'), ('name', 'tennis tournament')]
1


In [15]:
### insert the result of TASK 1 in the file
og_uri = "http://www.wikidata.org/entity/Q13219666'"
og_name = "tennis tournamente"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"2", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 0_4
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_4.json
JSON object updated


`wd:Q13219666` is the node of the tennis tournament in wikidata

## Task 3 
Identify the four Grand Slam of tennis (TIP: One of these four is Wimbledon)

In [8]:
# some queries above I found Q47345468 that is tennis tournament edition. Let see some instances that contains Wimbledon
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
    ?x wdt:P31 wd:Q47345468 .
    ?x <http://schema.org/name> ?name .
    FILTER(REGEX(?name, "imbledon")).
}
"""

print("Results")
x= run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q268871'), ('name', '2004 Wimbledon Championships')]
[('x', 'http://www.wikidata.org/entity/Q173961'), ('name', '1978 Wimbledon Championships')]
[('x', 'http://www.wikidata.org/entity/Q925026'), ('name', '1971 Wimbledon Championships')]
[('x', 'http://www.wikidata.org/entity/Q15854765'), ('name', '2014 Wimbledon Championships')]
[('x', 'http://www.wikidata.org/entity/Q30085309'), ('name', '2018 Wimbledon Championships')]
[('x', 'http://www.wikidata.org/entity/Q745901'), ('name', '1974 Wimbledon Championships')]
[('x', 'http://www.wikidata.org/entity/Q655368'), ('name', '1955 Wimbledon Championships')]
[('x', 'http://www.wikidata.org/entity/Q655378'), ('name', '1956 Wimbledon Championships')]
[('x', 'http://www.wikidata.org/entity/Q657950'), ('name', '1957 Wimbledon Championships')]
[('x', 'http://www.wikidata.org/entity/Q657962'), ('name', '1958 Wimbledon Championships')]
[('x', 'http://www.wikidata.org/entity/Q2296335'), ('name', '1881 Wi

In [9]:
# I found all the Wimbledon edition, let see their class
queryString = """
SELECT DISTINCT ?inst ?iName
WHERE { 
    ?x wdt:P31 wd:Q47345468 .
    ?x sc:name ?name .
    ?x wdt:P31 ?inst.
    FILTER(REGEX(?name, "imbledon")).
    ?inst sc:name ?iName.
}
"""

print("Results")
x= run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q47345468'), ('iName', 'tennis tournament edition')]
[('inst', 'http://www.wikidata.org/entity/Q41520'), ('iName', 'Wimbledon Championships')]
2


In [10]:
# I found the wimbledon championships, let see if it is instance of Grand Slam
queryString = """
SELECT DISTINCT ?inst ?name
WHERE { 
    wd:Q41520 wdt:P31 ?inst.
    ?inst sc:name ?name.
}
"""

print("Results")
x = run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q18608583'), ('name', 'recurring sporting event')]
1


In [11]:
# I found the wimbledon championships, let see if it is subclass of Grand Slam
queryString = """
SELECT DISTINCT ?inst ?name
WHERE { 
    wd:Q41520 wdt:P279 ?inst.
    ?inst sc:name ?name.
}
"""

print("Results")
x = run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q13219666'), ('name', 'tennis tournament')]
1


In [12]:
# let see if it is related with something named Grand Slam
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
    wd:Q41520 ?prop ?x.
    ?x sc:name ?name.
    FILTER(REGEX(?name,"[S,s]lam")).
}
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q102113'), ('name', 'Grand Slam of tennis')]
1


`wd:Q102113` is the node of the Grand Slam of tennis in wikidata. Check the instance of this or if there exists part of this.

In [13]:
# let see the properties of the Grand Slam of tennis
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
    wd:Q102113 ?prop ?x.
    ?prop sc:name ?name.
}
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('prop', 'http://www.wikidata.org/prop/direct/P2354'), ('name', 'has list')]
[('prop', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('prop', 'http://www.wikidata.org/prop/direct/P2924'), ('name', 'Great Russian Encyclopedia Online ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('prop', 'http://www.wikidata.org/prop/direct/P3222'), ('name', 'NE.se ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('prop', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('prop', 'http://www.wikidata.org/prop/direct/P641'), ('n

In [14]:
# let see the properties of the Grand Slam of tennis
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
    wd:Q102113 wdt:P527 ?x.
    ?x sc:name ?name.
}
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q2223649'), ('name', 'French cheese')]
[('x', 'http://www.wikidata.org/entity/Q123577'), ('name', 'US Open')]
[('x', 'http://www.wikidata.org/entity/Q41520'), ('name', 'Wimbledon Championships')]
[('x', 'http://www.wikidata.org/entity/Q60874'), ('name', 'Australian Open')]
4


Very funny this French Cheese!

In [15]:
# let see the properties of the Grand Slam of tennis on the other way
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
    ?x ?prop wd:Q102113.
    ?prop sc:name ?name.
}
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1423'), ('name', 'template has topic')]
[('prop', 'http://www.wikidata.org/prop/direct/P2522'), ('name', 'victory')]
[('prop', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('prop', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('prop', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('prop', 'http://www.wikidata.org/prop/direct/P971'), ('name', 'category combines topics')]
6


In [20]:
# use the property part of
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
    ?x wdt:P361 wd:Q102113.
    ?x sc:name ?name.
}
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q43605'), ('name', 'French Open')]
[('x', 'http://www.wikidata.org/entity/Q123577'), ('name', 'US Open')]
[('x', 'http://www.wikidata.org/entity/Q41520'), ('name', 'Wimbledon Championships')]
[('x', 'http://www.wikidata.org/entity/Q60874'), ('name', 'Australian Open')]
4


In [21]:
obj = [{"uri":r[0][1],"name":r[1][1]} for r in x]
evaluation.add_result(evaluation.get_index_workflow(pt),"3", evaluation.TYPE_SET ,"uri", obj)

The index of this workflow is: 0_4
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_4.json
JSON object updated


Found the four Grand Slams of tennis
- `Q43605` is the French Open
- `Q41520` is the Wimbledon Championship
- `Q60874` is the Australian Open
- `Q123577` is the US Open

## Task 4
Find the three different type of surfaces in tennis

In [17]:
# see the properties of the Grand Slams events
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
    ?x wdt:P361 wd:Q102113.
    ?x ?prop ?y.
    ?prop sc:name ?name.
}
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('prop', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('prop', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('prop', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('prop', 'http://www.wikidata.org/prop/direct/P1687'), ('name', 'Wikidata property')]
[('prop', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('prop', 'http://www.wikidata.org/prop/direct/P1814'), ('name', 'name in kana')]
[('prop', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('prop', 'http://www.

In [22]:
# use the property surface played on P765
queryString = """
SELECT DISTINCT ?y ?name
WHERE { 
    ?x wdt:P361 wd:Q102113.
    ?x wdt:P765 ?y.
    ?y sc:name ?name.
}
"""

print("Results")
x = run_query(queryString)

Results
[('y', 'http://www.wikidata.org/entity/Q1330871'), ('name', 'hardcourt')]
[('y', 'http://www.wikidata.org/entity/Q1151291'), ('name', 'grass court')]
[('y', 'http://www.wikidata.org/entity/Q198718'), ('name', 'clay court')]
3


In [23]:
obj = [{"uri":r[0][1],"name":r[1][1]} for r in x]
evaluation.add_result(evaluation.get_index_workflow(pt),"4", evaluation.TYPE_SET ,"uri", obj)

The index of this workflow is: 0_4
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_4.json
JSON object updated


## Task 5 
Find which are the tournament played in the 2018 Wimbledon Championships edition with the related winners (TIP: men and women do not play together, there are two distinct tournament )

In [19]:
# find the editions of Wimbledon
queryString = """
SELECT DISTINCT ?x ?name 
WHERE { 
    ?x wdt:P31 wd:Q41520.
    ?x sc:name ?name.
}
ORDER BY DESC (?name)
LIMIT 5
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q107283026'), ('name', '2022 Wimbledon Championships')]
[('x', 'http://www.wikidata.org/entity/Q92032430'), ('name', '2021 Wimbledon Championships')]
[('x', 'http://www.wikidata.org/entity/Q54093953'), ('name', '2019 Wimbledon Championships')]
[('x', 'http://www.wikidata.org/entity/Q30085309'), ('name', '2018 Wimbledon Championships')]
[('x', 'http://www.wikidata.org/entity/Q25344121'), ('name', '2017 Wimbledon Championships')]
5


In [20]:
# explore the editions of 2018
queryString = """
SELECT DISTINCT ?prop ?name 
WHERE { 
    wd:Q30085309 ?prop ?x.
    ?prop sc:name ?name.
}
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P6880'), ('name', 'ATP tennis tournament edition ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('prop', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('prop', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('prop', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('prop', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
[('prop', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('prop', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('prop', 'http://www.wikidata.org/prop/direct/P393'), ('name', 'edition number')]
[('prop', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('prop', 'http://www.wikidata.org/prop/direct

In [21]:
# use th property has part P527
queryString = """
SELECT DISTINCT ?x ?name 
WHERE { 
    wd:Q30085309 wdt:P527 ?x.
    ?x sc:name ?name.
}
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q30098268'), ('name', "2018 Wimbledon Championships – Women's Singles")]
[('x', 'http://www.wikidata.org/entity/Q32358401'), ('name', '2018 Wimbledon Championships – Mixed Doubles')]
[('x', 'http://www.wikidata.org/entity/Q32357790'), ('name', "2018 Wimbledon Championships – Men's Doubles")]
[('x', 'http://www.wikidata.org/entity/Q32356704'), ('name', "2018 Wimbledon Championships – Women's Doubles")]
[('x', 'http://www.wikidata.org/entity/Q55381839'), ('name', "2018 Wimbledon Championships – Girls' Singles")]
[('x', 'http://www.wikidata.org/entity/Q54961494'), ('name', "2018 Wimbledon Championships – Men's Singles")]
[('x', 'http://www.wikidata.org/entity/Q55410290'), ('name', "2018 Wimbledon Championships – Girls' Doubles")]
[('x', 'http://www.wikidata.org/entity/Q55410361'), ('name', "2018 Wimbledon Championships – Boys' Doubles")]
[('x', 'http://www.wikidata.org/entity/Q55455404'), ('name', "2018 Wimbledon Championships – Wheelchair Wo

In [22]:
# use th property has part P527
queryString = """
SELECT DISTINCT ?x ?name 
WHERE { 
    ?x wdt:P361 wd:Q30085309 .
    ?x sc:name ?name.
}
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q30098268'), ('name', "2018 Wimbledon Championships – Women's Singles")]
[('x', 'http://www.wikidata.org/entity/Q32358401'), ('name', '2018 Wimbledon Championships – Mixed Doubles')]
[('x', 'http://www.wikidata.org/entity/Q55639017'), ('name', '2018 Wimbledon Championships – Wheelchair Quad Doubles')]
[('x', 'http://www.wikidata.org/entity/Q55455418'), ('name', "2018 Wimbledon Championships – Wheelchair Men's Doubles")]
[('x', 'http://www.wikidata.org/entity/Q32357790'), ('name', "2018 Wimbledon Championships – Men's Doubles")]
[('x', 'http://www.wikidata.org/entity/Q32356704'), ('name', "2018 Wimbledon Championships – Women's Doubles")]
[('x', 'http://www.wikidata.org/entity/Q55381839'), ('name', "2018 Wimbledon Championships – Girls' Singles")]
[('x', 'http://www.wikidata.org/entity/Q54961494'), ('name', "2018 Wimbledon Championships – Men's Singles")]
[('x', 'http://www.wikidata.org/entity/Q55410290'), ('name', "2018 Wimbledon Champions

In [23]:
# see the properties of each event
queryString = """
SELECT DISTINCT ?prop ?name 
WHERE { 
    ?x wdt:P361 wd:Q30085309 .
    ?x ?prop ?y.
    ?prop sc:name ?name.
}
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('prop', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('prop', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('prop', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('prop', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class')]
[('prop', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('prop', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('prop', 'http://www.wikidata.org/prop/direct/P585'), ('name', 'point in time')]
[('prop', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
10


In [24]:
# see the properties of each event
queryString = """
SELECT DISTINCT ?x ?name ?y ?people
WHERE { 
    ?x wdt:P361 wd:Q30085309 .
    ?x wdt:P1346 ?y.
    ?x sc:name ?name.
    ?y sc:name ?people.
}
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q55455423'), ('name', "2018 Wimbledon Championships – Wheelchair Women's Doubles"), ('y', 'http://www.wikidata.org/entity/Q16224602'), ('people', 'Yui Kamiji')]
[('x', 'http://www.wikidata.org/entity/Q55455418'), ('name', "2018 Wimbledon Championships – Wheelchair Men's Doubles"), ('y', 'http://www.wikidata.org/entity/Q26951148'), ('people', 'Alfie Hewett')]
[('x', 'http://www.wikidata.org/entity/Q32356704'), ('name', "2018 Wimbledon Championships – Women's Doubles"), ('y', 'http://www.wikidata.org/entity/Q3720044'), ('people', 'Kateřina Siniaková')]
[('x', 'http://www.wikidata.org/entity/Q54961494'), ('name', "2018 Wimbledon Championships – Men's Singles"), ('y', 'http://www.wikidata.org/entity/Q5812'), ('people', 'Novak Djokovic')]
[('x', 'http://www.wikidata.org/entity/Q30098268'), ('name', "2018 Wimbledon Championships – Women's Singles"), ('y', 'http://www.wikidata.org/entity/Q77178'), ('people', 'Angelique Kerber')]
[('x', 'http://ww

In [25]:
# add the elements to the JSON gt
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    c_uri = i[2][1]
    c_name = i[3][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = ["uri","name"]
    obj["uri"]= c_uri
    obj["name"]= c_name
    objs.append(obj)

evaluation.add_result(evaluation.get_index_workflow(pt),"5", evaluation.TYPE_REFERRED ,["uri","name"], objs)

The index of this workflow is: 0_4
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_4.json
JSON object updated


## Task 6
Consider tennis players born after 1975, find who are the top 10 Grand Slams winner

In [24]:
# find the property date of birth
# see the properties of the instances of the Grand Slam
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
    wd:Q1426 ?prop ?x.
    ?prop sc:name ?name.
    FILTER(REGEX(?name,"birth")).
}
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P19'), ('name', 'place of birth')]
[('prop', 'http://www.wikidata.org/prop/direct/P569'), ('name', 'date of birth')]
2


In [26]:
# see the properties of the instances of the Grand Slam
queryString = """
SELECT DISTINCT ?people ?name COUNT(?tournament) AS ?win
WHERE { 
    #Grand slams
    ?x wdt:P361 wd:Q102113.
    ?edition wdt:P31 ?x.
    ?tournament wdt:P361 ?edition.
    ?tournament wdt:P1346 ?people.
    ?people wdt:P569 ?birth.
    ?people sc:name ?name.
    FILTER (year(?birth) > 1975).
}
GROUP BY ?people ?name
ORDER BY DESC (?win)
LIMIT 10
"""

print("Results")
x = run_query(queryString)

Results
[('people', 'http://www.wikidata.org/entity/Q11459'), ('name', 'Serena Williams'), ('win', '39')]
[('people', 'http://www.wikidata.org/entity/Q959958'), ('name', 'Shingo Kunieda'), ('win', '32')]
[('people', 'http://www.wikidata.org/entity/Q134720'), ('name', 'Martina Hingis'), ('win', '25')]
[('people', 'http://www.wikidata.org/entity/Q16224602'), ('name', 'Yui Kamiji'), ('win', '22')]
[('people', 'http://www.wikidata.org/entity/Q1426'), ('name', 'Roger Federer'), ('win', '22')]
[('people', 'http://www.wikidata.org/entity/Q259606'), ('name', 'Esther Vergeer'), ('win', '21')]
[('people', 'http://www.wikidata.org/entity/Q10132'), ('name', 'Rafael Nadal'), ('win', '20')]
[('people', 'http://www.wikidata.org/entity/Q5812'), ('name', 'Novak Djokovic'), ('win', '20')]
[('people', 'http://www.wikidata.org/entity/Q11578'), ('name', 'Venus Williams'), ('win', '20')]
[('people', 'http://www.wikidata.org/entity/Q53340'), ('name', 'Bob Bryan'), ('win', '18')]
10


In [27]:
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    victories = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= victories
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"6", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 0_4
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_4.json
JSON object updated


## Task 7
Consider the country for sport property of the tennis player (NO Country of citizenship). Find which are the countries with at least one player who won at least one edition of each Grand Slam

In [26]:
# find the property country for sport 
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
    wd:Q1426 ?prop ?x.
    ?prop sc:name ?name.
    FILTER(REGEX(?name,"sport")).
}
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]
[('prop', 'http://www.wikidata.org/prop/direct/P2416'), ('name', 'sports discipline competed in')]
[('prop', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('prop', 'http://www.wikidata.org/prop/direct/P6758'), ('name', 'supported sports team')]
4


In [26]:
# see the properties of the instances of the Grand Slam
queryString = """
SELECT DISTINCT ?people ?name COUNT(DISTINCT ?x) AS ?win
WHERE { 
    #Grand slams
    ?x wdt:P361 wd:Q102113.
    ?edition wdt:P31 ?x.
    ?tournament wdt:P361 ?edition.
    ?tournament wdt:P1346 ?people.
    ?people wdt:P1532 ?country.
    ?people sc:name ?name.
}
GROUP BY ?people ?name
HAVING (COUNT(DISTINCT ?x) >3 )
"""

print("Results")
x = run_query(queryString)

Results
[('people', 'http://www.wikidata.org/entity/Q10118'), ('name', 'Victoria Azarenka'), ('win', '4')]
[('people', 'http://www.wikidata.org/entity/Q53338'), ('name', 'Louise Brough'), ('win', '4')]
[('people', 'http://www.wikidata.org/entity/Q11578'), ('name', 'Venus Williams'), ('win', '4')]
[('people', 'http://www.wikidata.org/entity/Q1426'), ('name', 'Roger Federer'), ('win', '4')]
[('people', 'http://www.wikidata.org/entity/Q5812'), ('name', 'Novak Djokovic'), ('win', '4')]
[('people', 'http://www.wikidata.org/entity/Q2093033'), ('name', 'Pierre-Hugues Herbert'), ('win', '4')]
[('people', 'http://www.wikidata.org/entity/Q11459'), ('name', 'Serena Williams'), ('win', '4')]
[('people', 'http://www.wikidata.org/entity/Q270414'), ('name', 'Anne Smith'), ('win', '4')]
[('people', 'http://www.wikidata.org/entity/Q959958'), ('name', 'Shingo Kunieda'), ('win', '4')]
[('people', 'http://www.wikidata.org/entity/Q54527'), ('name', 'Billie Jean King'), ('win', '4')]
[('people', 'http://www

In [28]:
# see the properties of the instances of the Grand Slam
queryString = """
SELECT ?country ?name COUNT(DISTINCT ?people) AS ?people
WHERE{
    ?people wdt:P1532 ?country.
    {
        SELECT ?people COUNT(DISTINCT ?x) AS ?win
        WHERE { 
            #Grand slams
            ?x wdt:P361 wd:Q102113.
            ?edition wdt:P31 ?x.
            ?tournament wdt:P361 ?edition.
            ?tournament wdt:P1346 ?people.
        }
        GROUP BY ?people
        HAVING (COUNT(DISTINCT ?x) >3 )
    }
    ?country sc:name ?name.
}
GROUP BY ?country ?name
"""

print("Results")
x = run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('people', '2')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil'), ('people', '1')]
[('country', 'http://www.wikidata.org/entity/Q408'), ('name', 'Australia'), ('people', '2')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy'), ('people', '1')]
[('country', 'http://www.wikidata.org/entity/Q28'), ('name', 'Hungary'), ('people', '1')]
[('country', 'http://www.wikidata.org/entity/Q33946'), ('name', 'Czechoslovakia'), ('people', '3')]
[('country', 'http://www.wikidata.org/entity/Q403'), ('name', 'Serbia'), ('people', '1')]
[('country', 'http://www.wikidata.org/entity/Q17'), ('name', 'Japan'), ('people', '2')]
[('country', 'http://www.wikidata.org/entity/Q213'), ('name', 'Czech Republic'), ('people', '4')]
[('country', 'http://www.wikidata.org/entity/Q215'), ('name', 'Slovenia'), ('people', '1')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany'),

In [29]:
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    victories = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= victories
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"7", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 0_4
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_4.json
JSON object updated


## Task 8
Consider the country for sport property of the tennis player (NO Country of citizenship). Find which are the countries with at least one player who was ranked 1 in the tennis ranking at least once

In [46]:
# find the class of country
queryString = """
SELECT DISTINCT ?prop ?name
WHERE{
    wd:Q1426 ?prop ?x.
    ?prop sc:name ?name.
}
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('prop', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('prop', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('prop', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1284'), ('name', 'Munzinger person ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('prop', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('prop', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('prop', 'http://www.wikidat

In [47]:
# find the class of country
queryString = """
SELECT DISTINCT ?x
WHERE{
    wd:Q1426 wdt:P1352 ?x.
}
"""

print("Results")
x = run_query(queryString)

Results
[('x', '1')]
[('x', '24')]
2


In [30]:
# see the properties of the instances of the Grand Slam
queryString = """
SELECT DISTINCT ?country ?name
WHERE { 
    #Grand slams
    ?people wdt:P1532 ?country;
            wdt:P1352 ?ranking;
            wdt:P106 wd:Q10833314.
    ?country sc:name ?name.
    FILTER(?ranking = 1)
}
GROUP BY ?country ?name
"""

print("Results")
x = run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q212'), ('name', 'Ukraine')]
[('country', 'http://www.wikidata.org/entity/Q17'), ('name', 'Japan')]
[('country', 'http://www.wikidata.org/entity/Q215'), ('name', 'Slovenia')]
[('country', 'http://www.wikidata.org/entity/Q34'), ('name', 'Sweden')]
[('country', 'http://www.wikidata.org/entity/Q39'), ('name', 'Switzerland')]
[('country', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('country', 'http://www.wikidata.org/entity/Q224'), ('name', 'Croatia')]
[('country', 'http://www.wikidata.org/entity/Q33'), ('name', 'Finland')]
[('country', 'http://www.wikidata.org/entity/Q36'), ('name', 'Poland')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('name', 'Spain')]
[('country', 'http://www.wikidata.org/entity/Q408'), ('name', 'Australia')]
[('country', 'http://www.wikidata.org/entity/Q33946'), ('name', 'Czechoslovakia')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil')]
[('country', 'http://www.wik

In [31]:
obj = [{"uri":r[0][1],"name":r[1][1]} for r in x]
evaluation.add_result(evaluation.get_index_workflow(pt),"8", evaluation.TYPE_SET ,"uri", obj)

The index of this workflow is: 0_4
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_4.json
JSON object updated
